In [31]:
import numpy as np
import pandas as pd

import pickle
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import sys
import os


In [32]:
notebook_path = os.path.abspath(".")
sys.path.append(os.path.abspath(os.path.join(notebook_path, '..', 'src')))
print(sys.path)


['c:\\Users\\Jocawl\\anaconda3\\python312.zip', 'c:\\Users\\Jocawl\\anaconda3\\DLLs', 'c:\\Users\\Jocawl\\anaconda3\\Lib', 'c:\\Users\\Jocawl\\anaconda3', 'e:\\hackABoss\\pfb-coches\\.venv', '', 'e:\\hackABoss\\pfb-coches\\.venv\\Lib\\site-packages', 'e:\\hackABoss\\pfb-coches\\.venv\\Lib\\site-packages\\win32', 'e:\\hackABoss\\pfb-coches\\.venv\\Lib\\site-packages\\win32\\lib', 'e:\\hackABoss\\pfb-coches\\.venv\\Lib\\site-packages\\Pythonwin', 'e:\\hackABoss\\pfb-coches\\src', 'e:\\hackABoss\\pfb-coches\\src']


In [33]:
from limpieza import transforma_ML
import encoding_func
import ocasionDataBase as odb

In [34]:
ruta_config: str = "../.streamlit/secrets.toml"
dict_config: dict = odb.load_config(ruta_config, "database_user")

db: odb.OcasionDataBase = odb.OcasionDataBase(dict_config)

df: pd.DataFrame = pd.DataFrame(db.obtener_coches_venta())
df.head(3)

,referencia,peninsula_baleares,combustible,potencia,cambio_automatico,carroceria,kilometraje,distintivo_ambiental,color,garantia,...,consumo,modelo_titulo,marca_sola,antiguedad,precio,mes_matricula,anio_matricula,concesionario,url,ruta_imagen
0,3660837,1,Diésel,245,1,Berlina,179000,B,OTROS,16,...,5.89,Serie 7 730 D**COMO NUEVO**PACK M**FULL**NACIONAL,BMW,14,19900.0,6,2011,BETULO GLOBAL,https://www.autocasion.com/coches-segunda-mano...,img/coches/33ba47613800fe88de63bddd9cdb1adf.jpeg
1,3938343,1,Gasolina,272,1,Deportivo o coupé,148000,C,NEGRO,12,...,8.00,Serie 3 330xi Coupé Aut.,BMW,18,13500.0,1,2007,ESTANY AUTOMOCIÓ,https://www.autocasion.com/coches-segunda-mano...,img/coches/7bec3bde6858ae760b925671dc7734bf095...
2,4683324,1,Gasolina,262,1,Monovolumen,46400,C,BLANCO,16,...,5.89,Sienna 3.5L V6 AWD XLE,TOYOTA,6,71800.0,5,2019,EUROPEAN AUTO BROKER,https://www.autocasion.com/coches-segunda-mano...,img/coches/4683324-0-2934626031657bffc934c466....


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19890 entries, 0 to 19889
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   referencia            19890 non-null  int64         
 1   peninsula_baleares    19890 non-null  int64         
 2   combustible           19890 non-null  object        
 3   potencia              19890 non-null  int64         
 4   cambio_automatico     19890 non-null  int64         
 5   carroceria            19890 non-null  object        
 6   kilometraje           19890 non-null  int64         
 7   distintivo_ambiental  19890 non-null  object        
 8   color                 19890 non-null  object        
 9   garantia              19890 non-null  int64         
 10  vendedor_profesional  19890 non-null  int64         
 11  plazas                19890 non-null  int64         
 12  puertas               19890 non-null  int64         
 13  certificado     

In [36]:
df_modelo = df.drop(columns=["modelo_titulo", "referencia", "url", "ruta_imagen", "fecha_extraccion", "garantia"])
df_modelo = transforma_ML.normalizar(df_modelo, ["kilometraje", "potencia", "precio"])

In [37]:
#Elimino filas con NaN en precio
df_modelo = df_modelo.dropna(subset=["precio"])
df_modelo.isna().sum()


peninsula_baleares       0
combustible              0
potencia                 0
cambio_automatico        0
carroceria               0
kilometraje              0
distintivo_ambiental     0
color                    0
vendedor_profesional     0
plazas                   0
puertas                  0
certificado              0
consumo                  3
marca_sola               0
antiguedad               0
precio                   0
mes_matricula            0
anio_matricula           0
concesionario           32
dtype: int64

In [38]:
#encoding_func.generar_pickle_label_encoding(df_modelo,["combustible", "distintivo_ambiental", "carroceria", "color"])
encoding_func.generar_pickle_onehot_encoding(df_modelo, ["cambio_automatico"]) #["cambio_automatico", "vendedor_profesional", "certificado", "peninsula_y_baleares"])

Pickle OHE cambio_automatico_encoder.pickle generado.


In [39]:
with open(f"../bin/cambio_automatico_encoder.pickle", "rb") as file:
    cambio_automatico_encoder = pickle.load(file)
    

In [40]:
#df_modelo["combustible"] = combustible_encoder.transform(df_modelo["combustible"].astype(str))
#df_modelo["carroceria"] = carroceria_encoder.transform(df_modelo["carroceria"].astype(str))
#df_modelo["color"] = color_encoder.transform(df_modelo["color"].astype(str))
#df_modelo["distintivo_ambiental"] = distintivo_ambiental_encoder.transform(df_modelo["distintivo_ambiental"].astype(str))


In [41]:
df_modelo["cambio_automatico"] = cambio_automatico_encoder.transform(df_modelo[["cambio_automatico"]])


In [42]:
df_modelo.columns

Index(['peninsula_baleares', 'combustible', 'potencia', 'cambio_automatico',
       'carroceria', 'kilometraje', 'distintivo_ambiental', 'color',
       'vendedor_profesional', 'plazas', 'puertas', 'certificado', 'consumo',
       'marca_sola', 'antiguedad', 'precio', 'mes_matricula', 'anio_matricula',
       'concesionario'],
      dtype='object')

In [43]:
df_modelo = df_modelo[['kilometraje', 'cambio_automatico', 'potencia', 'marca_sola', 'antiguedad', 'precio']]

In [44]:
TARGET = "precio"
X_train, X_test, y_train, y_test = encoding_func.dividir_dataframe(df_modelo, TARGET)

encoding_func.generar_pickle_target_encoding(X_train, y_train, [ "marca_sola"], TARGET)

Pickle TE marca_sola_precio_encoder.pickle generado.


In [45]:
with open(f"../bin/marca_sola_precio_encoder.pickle", "rb") as file:
    marca_sola_precio_encoder = pickle.load(file)



In [46]:
X_train["marca_sola"] = marca_sola_precio_encoder.transform(X_train["marca_sola"])

X_test["marca_sola"] = marca_sola_precio_encoder.transform(X_test["marca_sola"])

In [47]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15888 entries, 11818 to 15815
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   kilometraje        15888 non-null  float64
 1   cambio_automatico  15888 non-null  float64
 2   potencia           15888 non-null  float64
 3   marca_sola         15888 non-null  float64
 4   antiguedad         15888 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 744.8 KB


In [48]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3973 entries, 15224 to 9012
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   kilometraje        3973 non-null   float64
 1   cambio_automatico  3973 non-null   float64
 2   potencia           3973 non-null   float64
 3   marca_sola         3973 non-null   float64
 4   antiguedad         3973 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 186.2 KB


In [49]:
X_train.head(3)

,kilometraje,cambio_automatico,potencia,marca_sola,antiguedad
11818,9.588777,0.0,4.875197,9.757617,1
8769,11.365086,0.0,4.615121,9.854138,22
3254,2.397895,1.0,5.416100,9.562919,1


In [50]:
scaler = MinMaxScaler()
scaler.set_output(transform="pandas")

X_train_escalado = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_escalado = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

with open(f"../bin/min_max_scaler.pickle", "wb") as file:
    pickle.dump(scaler, file)

In [52]:
X_train_escalado

,kilometraje,cambio_automatico,potencia,marca_sola,antiguedad
11818,0.748197,0.0,0.519865,0.181552,0.033333
8769,0.886800,0.0,0.465288,0.235092,0.733333
3254,0.187104,1.0,0.633375,0.073555,0.033333
16464,0.866629,1.0,0.506642,0.302464,0.100000
5659,0.867135,0.0,0.494346,0.310013,0.266667
...,...,...,...,...,...
11297,0.854476,0.0,0.465288,0.233676,0.166667
11977,0.746583,1.0,0.529263,0.181552,0.033333
5396,0.916986,1.0,0.627703,0.302464,0.200000
860,0.812817,0.0,0.454633,0.000000,0.100000
